In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

43287


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数
43282,2022-07-20,47,沖縄県,5160,305118,0,497,1655.97
43283,2022-07-21,47,沖縄県,5250,310368,0,497,1770.79
43284,2022-07-22,47,沖縄県,4654,315022,3,500,1852.02
43285,2022-07-23,47,沖縄県,5297,320319,3,503,1946.94
43286,2022-07-24,47,沖縄県,4625,324944,0,503,1978.29


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数
42,静岡県,20262,228441,435,41204.54
43,香川県,34077,62040,138,43312.04
44,高知県,35919,38385,118,36879.60
45,鳥取県,28551,24414,21,28916.83
46,鹿児島県,42366,126124,221,52261.46


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数
43280,2022-07-18,47,沖縄県,1990,297903,0,497,1638.18,24040.0
43281,2022-07-19,47,沖縄県,2055,299958,0,497,1544.08,22659.0
43282,2022-07-20,47,沖縄県,5160,305118,0,497,1655.97,24301.0
43283,2022-07-21,47,沖縄県,5250,310368,0,497,1770.79,25986.0
43284,2022-07-22,47,沖縄県,4654,315022,3,500,1852.02,27178.0
43285,2022-07-23,47,沖縄県,5297,320319,3,503,1946.94,28571.0
43286,2022-07-24,47,沖縄県,4625,324944,0,503,1978.29,29031.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
43277,2022-07-15,47,沖縄県,3462,287844,0,497,1356.07,19900.0,-0.028892,0.293375,0.432354
43278,2022-07-16,47,沖縄県,3904,291748,0,497,1450.51,21286.0,0.127672,0.305645,0.550437
43279,2022-07-17,47,沖縄県,4165,295913,0,497,1566.84,22993.0,0.066855,0.318599,0.694467
43280,2022-07-18,47,沖縄県,1990,297903,0,497,1638.18,24040.0,-0.522209,0.332049,1.110286
43281,2022-07-19,47,沖縄県,2055,299958,0,497,1544.08,22659.0,0.032663,-0.040955,-0.401921
43282,2022-07-20,47,沖縄県,5160,305118,0,497,1655.97,24301.0,1.510949,0.171485,0.466742
43283,2022-07-21,47,沖縄県,5250,310368,0,497,1770.79,25986.0,0.017442,0.172069,0.472651
43284,2022-07-22,47,沖縄県,4654,315022,3,500,1852.02,27178.0,-0.113524,0.159978,0.344310
43285,2022-07-23,47,沖縄県,5297,320319,3,503,1946.94,28571.0,0.138161,0.161477,0.356814
43286,2022-07-24,47,沖縄県,4625,324944,0,503,1978.29,29031.0,-0.126864,0.133803,0.110444


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
920,2022-07-24,北海道,4072,2,21897.0,-0.121657,0.187796,1.221495
1841,2022-07-24,青森県,1308,0,8998.0,-0.174763,0.119286,0.595122
2762,2022-07-24,岩手県,911,1,5236.0,-0.133206,0.129905,0.644404
3683,2022-07-24,宮城県,2017,0,12059.0,-0.086504,0.463594,1.305143
4604,2022-07-24,秋田県,640,0,4270.0,-0.224242,0.289930,1.071197


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,22104,109388,324,40895.07,2022-07-24,2185,1,11140.0,0.094689,0.190387,0.845439
1,京都府,23946,254909,764,66025.80,2022-07-24,4687,0,21759.0,0.115687,0.240738,0.899878
2,佐賀県,37761,77538,120,63541.08,2022-07-24,756,1,8727.0,-0.482192,0.091733,0.145455
3,兵庫県,25788,537761,2274,65575.60,2022-07-24,9336,0,48396.0,-0.016849,0.239336,0.676724
4,北海道,921,420850,2115,54764.03,2022-07-24,4072,2,21897.0,-0.121657,0.187796,1.221495


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,22104,109388,324,40895.07,2022-07-24,2185,1,11140.0,0.094689,0.190387,0.845439,1813859
1,京都府,23946,254909,764,66025.80,2022-07-24,4687,0,21759.0,0.115687,0.240738,0.899878,2545899
2,佐賀県,37761,77538,120,63541.08,2022-07-24,756,1,8727.0,-0.482192,0.091733,0.145455,823810
3,兵庫県,25788,537761,2274,65575.60,2022-07-24,9336,0,48396.0,-0.016849,0.239336,0.676724,5549568
4,北海道,921,420850,2115,54764.03,2022-07-24,4072,2,21897.0,-0.121657,0.187796,1.221495,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,921,420850,2115,54764.03,2022-07-24,4072,2,21897.0,-0.121657,0.187796,1.221495,5267762,798.916124,7989.161242,415.679372
41,青森県,1842,81602,123,43651.81,2022-07-24,1308,0,8998.0,-0.174763,0.119286,0.595122,1275783,639.622883,6396.228826,705.292358
19,岩手県,2763,48301,104,26326.98,2022-07-24,911,1,5236.0,-0.133206,0.129905,0.644404,1235517,390.937559,3909.375589,423.790203
11,宮城県,3684,114750,205,32955.99,2022-07-24,2017,0,12059.0,-0.086504,0.463594,1.305143,2292385,500.570367,5005.703667,526.046018
36,秋田県,4605,39049,74,26907.49,2022-07-24,640,0,4270.0,-0.224242,0.289930,1.071197,985416,396.269190,3962.691899,433.319532
15,山形県,5526,36912,101,23016.44,2022-07-24,543,0,3526.0,-0.095000,0.189310,1.163347,1082296,341.052725,3410.527249,325.788878
35,福島県,6447,78146,229,28555.44,2022-07-24,1070,0,6518.0,-0.125102,0.136930,0.783333,1881981,415.232672,4152.326724,346.337184
38,茨城県,7368,187507,462,44129.64,2022-07-24,2757,1,12782.0,0.082025,0.170827,1.350384,2921436,641.831620,6418.316198,437.524560
27,栃木県,8289,112040,285,38481.17,2022-07-24,1914,0,10883.0,-0.104771,0.366210,1.246479,1965516,570.028430,5700.284302,553.696841
37,群馬県,9210,118351,331,40663.69,2022-07-24,1823,0,11340.0,-0.127751,0.154793,0.826653,1969439,600.937627,6009.376274,575.798489


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count    4.700000e+01
mean     2.418726e+05
std      3.610413e+05
min      2.441400e+04
25%      6.215250e+04
50%      1.093880e+05
75%      2.115410e+05
max      1.941548e+06
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,11973,1941548,4623,92511.33,2022-07-24,28112,2,171798.0,-0.140253,0.126794,0.580214,13834925,1403.367203,14033.672029,1241.770375
1,大阪府,24867,1230773,5264,92875.26,2022-07-24,17439,2,113783.0,-0.224864,0.381678,0.614124,8849635,1390.761314,13907.613139,1285.736643
2,神奈川県,12894,927588,2272,67712.95,2022-07-24,12100,0,75256.0,-0.117819,0.075867,0.485209,9209442,1007.214118,10072.141179,817.161344
3,愛知県,21183,704167,2120,61893.05,2022-07-24,11514,3,71809.0,-0.197519,0.319960,0.850530,7575530,929.528363,9295.283630,947.907275
4,埼玉県,10131,682306,1669,61928.34,2022-07-24,12184,0,60854.0,-0.019160,0.179605,0.861005,7390054,923.276068,9232.760681,823.458124
5,福岡県,36840,569721,1311,73230.56,2022-07-24,10755,3,63554.0,-0.147579,0.166149,0.836264,5129841,1110.601674,11106.016736,1238.907795
6,千葉県,11052,555280,1825,59152.62,2022-07-24,8436,0,47330.0,-0.120425,0.134719,0.729752,6319772,878.639293,8786.392927,748.919423
7,兵庫県,25788,537761,2274,65575.60,2022-07-24,9336,0,48396.0,-0.016849,0.239336,0.676724,5549568,969.014165,9690.141647,872.067880
8,北海道,921,420850,2115,54764.03,2022-07-24,4072,2,21897.0,-0.121657,0.187796,1.221495,5267762,798.916124,7989.161242,415.679372
9,沖縄県,43287,324944,503,148360.00,2022-07-24,4625,0,29031.0,-0.126864,0.133803,0.110444,1481547,2193.275002,21932.750024,1959.505841


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count      47.000000
mean      720.282161
std       328.069670
min       341.052725
25%       514.512801
50%       639.622883
75%       858.938109
max      2193.275002
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,43287,324944,503,148360.00,2022-07-24,4625,0,29031.0,-0.126864,0.133803,0.110444,1481547,2193.275002,21932.750024,1959.505841
9,大阪府,24867,1230773,5264,92875.26,2022-07-24,17439,2,113783.0,-0.224864,0.381678,0.614124,8849635,1390.761314,13907.613139,1285.736643
26,東京都,11973,1941548,4623,92511.33,2022-07-24,28112,2,171798.0,-0.140253,0.126794,0.580214,13834925,1403.367203,14033.672029,1241.770375
34,福岡県,36840,569721,1311,73230.56,2022-07-24,10755,3,63554.0,-0.147579,0.166149,0.836264,5129841,1110.601674,11106.016736,1238.907795
30,熊本県,39603,161263,347,60662.13,2022-07-24,2822,2,21366.0,-0.242416,0.199499,0.203412,1769880,911.152169,9111.521685,1207.200488
2,佐賀県,37761,77538,120,63541.08,2022-07-24,756,1,8727.0,-0.482192,0.091733,0.145455,823810,941.212173,9412.121727,1059.346208
46,鹿児島県,42366,126124,221,52261.46,2022-07-24,2254,0,15462.0,-0.129730,0.102822,0.325882,1630146,773.697571,7736.975706,948.503999
24,愛知県,21183,704167,2120,61893.05,2022-07-24,11514,3,71809.0,-0.197519,0.319960,0.850530,7575530,929.528363,9295.283630,947.907275
12,宮崎県,41445,79367,155,48480.09,2022-07-24,1809,0,10251.0,0.014582,0.178321,0.763158,1095903,724.215556,7242.155556,935.393005
20,島根県,29472,37677,37,36481.59,2022-07-24,658,0,6037.0,-0.226792,0.162092,-0.050505,679324,554.624892,5546.248918,888.677568


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    47.000000
mean      0.200538
std       0.092530
min       0.062854
25%       0.134261
50%       0.179605
75%       0.240037
max       0.463594
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
11,宮城県,3684,114750,205,32955.99,2022-07-24,2017,0,12059.0,-0.086504,0.463594,1.305143,2292385,500.570367,5005.703667,526.046018
9,大阪府,24867,1230773,5264,92875.26,2022-07-24,17439,2,113783.0,-0.224864,0.381678,0.614124,8849635,1390.761314,13907.613139,1285.736643
31,石川県,15657,73471,200,43030.32,2022-07-24,1316,0,7151.0,-0.004539,0.370993,0.938144,1139612,644.701881,6447.018810,627.494270
27,栃木県,8289,112040,285,38481.17,2022-07-24,1914,0,10883.0,-0.104771,0.366210,1.246479,1965516,570.028430,5700.284302,553.696841
13,富山県,14736,50502,99,31992.70,2022-07-24,1099,0,5974.0,0.103414,0.364967,1.661017,1055999,478.239089,4782.390892,565.720233
23,愛媛県,34998,62265,145,30720.88,2022-07-24,1058,0,7593.0,-0.168239,0.356396,0.473538,1369131,454.777519,4547.775195,554.585354
16,山梨県,17499,43681,69,35494.16,2022-07-24,741,0,5215.0,-0.211702,0.321553,0.924675,826579,528.455235,5284.552354,630.913682
24,愛知県,21183,704167,2120,61893.05,2022-07-24,11514,3,71809.0,-0.197519,0.319960,0.850530,7575530,929.528363,9295.283630,947.907275
36,秋田県,4605,39049,74,26907.49,2022-07-24,640,0,4270.0,-0.224242,0.289930,1.071197,985416,396.269190,3962.691899,433.319532
40,長野県,18420,93613,206,30444.54,2022-07-24,1500,0,8858.0,-0.077491,0.269560,1.057613,2087307,448.486974,4484.869739,424.374565


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,各地の直近1週間の人口10万人あたりの感染者数,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
45,鳥取県,28551,24414,21,28916.83,2022-07-24,516,0,3315.0,-0.007692,0.062854,0.246377,561175,435.051455,4350.514545,590.724818
32,神奈川県,12894,927588,2272,67712.95,2022-07-24,12100,0,75256.0,-0.117819,0.075867,0.485209,9209442,1007.214118,10072.141179,817.161344
33,福井県,16578,49089,51,42496.09,2022-07-24,923,2,5207.0,-0.017039,0.082563,0.718808,780053,629.303393,6293.033935,667.518746
2,佐賀県,37761,77538,120,63541.08,2022-07-24,756,1,8727.0,-0.482192,0.091733,0.145455,823810,941.212173,9412.121727,1059.346208
29,滋賀県,23025,113574,225,54128.09,2022-07-24,1506,0,9116.0,0.211585,0.096301,0.519677,1420948,799.283295,7992.832954,641.543533
46,鹿児島県,42366,126124,221,52261.46,2022-07-24,2254,0,15462.0,-0.129730,0.102822,0.325882,1630146,773.697571,7736.975706,948.503999
10,奈良県,26709,113619,404,57569.55,2022-07-24,1535,1,9312.0,-0.021670,0.118414,0.300847,1353837,839.236924,8392.369244,687.822832
41,青森県,1842,81602,123,43651.81,2022-07-24,1308,0,8998.0,-0.174763,0.119286,0.595122,1275783,639.622883,6396.228826,705.292358
26,東京都,11973,1941548,4623,92511.33,2022-07-24,28112,2,171798.0,-0.140253,0.126794,0.580214,13834925,1403.367203,14033.672029,1241.770375
19,岩手県,2763,48301,104,26326.98,2022-07-24,911,1,5236.0,-0.133206,0.129905,0.644404,1235517,390.937559,3909.375589,423.790203


## グラフ化

In [19]:
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib notebook
#%matplotlib widget
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
for i,v in enumerate(chart_data["各地の感染者数_1日ごとの発表数"]):
    plt.text(v,i,"{:,}".format(v),va="center",fontsize=7)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
for i,v in enumerate(chart_data["直近1週間の10万人当たりの感染者数"]):
    plt.text(v,i,"{:,}".format(int(v)),va="center",fontsize=7)
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[37  2  5  1  0  1  0  1]
[  24414.    264055.75  503697.5   743339.25  982981.   1222622.75
 1462264.5  1701906.25 1941548.  ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([ 0.,  6., 13.,  4.,  6.,  3.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <BarContainer object of 7 artists>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([0., 0., 0., 0., 0., 0.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <BarContainer object of 6 artists>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,各地の直近1週間の人口10万人あたりの感染者数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
32225,2022-07-15,35,山口県,624,54206,0,181,233.60,3135.0,-0.071429,0.217711,1.151724
32226,2022-07-16,35,山口県,770,54976,0,181,268.32,3601.0,0.233974,0.244239,1.532895
32227,2022-07-17,35,山口県,772,55748,1,182,303.94,4079.0,0.002597,0.249309,1.625850
32228,2022-07-18,35,山口県,424,56172,0,182,320.48,4301.0,-0.450777,0.229616,1.099010
32229,2022-07-19,35,山口県,625,56797,1,183,327.78,4399.0,0.474057,0.067494,0.185958
32230,2022-07-20,35,山口県,1193,57990,1,184,378.52,5080.0,0.908800,0.201389,1.330078
32231,2022-07-21,35,山口県,1424,59414,3,187,434.56,5832.0,0.193630,0.184407,1.119048
32232,2022-07-22,35,山口県,1337,60751,3,190,487.68,6545.0,-0.061096,0.185884,1.142628
32233,2022-07-23,35,山口県,1195,61946,1,191,519.35,6970.0,-0.106208,0.137286,0.551948
32234,2022-07-24,35,山口県,1288,63234,1,192,557.80,7486.0,0.077824,0.148033,0.668394
